In [1]:
import os
import cv2
import glob

def yolo_to_xyxy(xc, yc, w, h, img_w, img_h):
    """Convert YOLO format to pixel coordinates."""
    x1 = int((xc - w / 2) * img_w)
    y1 = int((yc - h / 2) * img_h)
    x2 = int((xc + w / 2) * img_w)
    y2 = int((yc + h / 2) * img_h)
    return x1, y1, x2, y2

## crop the bounding boxes

In [3]:
def extract_cow_crops(video_path, label_dir, output_dir):

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(cap)
    print("****", fps)
    
    label_files = sorted(glob.glob(f"{label_dir}/*.txt"))
    os.makedirs(output_dir, exist_ok=True)

    cow_writers = {}  # track_id: VideoWriter

    for frame_idx, label_file in enumerate(label_files):
        ret, frame = cap.read()
        if not ret: break

        with open(label_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 7:
                    continue  # skip incomplete lines

                class_id = int(parts[0])
                xc, yc, w, h = map(float, parts[1:5])
                conf = float(parts[5])
                track_id = int(parts[6])

                # Convert to pixel coordinates
                x1 = int((xc - w / 2) * frame_w)
                y1 = int((yc - h / 2) * frame_h)
                x2 = int((xc + w / 2) * frame_w)
                y2 = int((yc + h / 2) * frame_h)

                # Safe clipping to frame size
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(frame_w - 1, x2), min(frame_h - 1, y2)

                crop = frame[y1:y2, x1:x2]
                if crop.size == 0:
                    continue

                cow_id = f"cow_{track_id}"
                cow_out_path = os.path.join(output_dir, cow_id + ".mp4")

                if cow_id not in cow_writers:
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                    cow_writers[cow_id] = cv2.VideoWriter(cow_out_path, fourcc, fps, (x2 - x1, y2 - y1))

                cow_writers[cow_id].write(crop)

    cap.release()
    for writer in cow_writers.values():
        writer.release()


### crop videos with padding to overcome varying bounding boxes problem 

In [1]:
import cv2
import os
import glob
import numpy as np

def center_pad(image, target_size=(640, 640)):
    h, w = image.shape[:2]
    th, tw = target_size
    # Skip if crop is too big
    if h > th or w > tw:
        return None
    top = (th - h) // 2
    bottom = th - h - top
    left = (tw - w) // 2
    right = tw - w - left
    return cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))


def extract_cow_crops_padded(video_path, label_dir, output_dir, target_size=(640, 640)):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Video: {video_path} | FPS: {fps}, Width: {frame_w}, Height: {frame_h}")    
    label_files = sorted(glob.glob(f"{label_dir}/*.txt"))
    os.makedirs(output_dir, exist_ok=True)

    cow_writers = {}

    for frame_idx, label_file in enumerate(label_files):
        ret, frame = cap.read()
        if not ret:
            break

        with open(label_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 7:
                    continue

                class_id = int(parts[0])
                xc, yc, w, h = map(float, parts[1:5])
                conf = float(parts[5])
                track_id = int(parts[6])

                x1 = int((xc - w / 2) * frame_w)
                y1 = int((yc - h / 2) * frame_h)
                x2 = int((xc + w / 2) * frame_w)
                y2 = int((yc + h / 2) * frame_h)

                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(frame_w - 1, x2), min(frame_h - 1, y2)

                crop = frame[y1:y2, x1:x2]
                if crop.size == 0:
                    continue

                padded_crop = center_pad(crop, target_size)
                cow_id = f"cow_{track_id}"
                cow_out_path = os.path.join(output_dir, cow_id + ".mp4")

                if cow_id not in cow_writers:
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                    cow_writers[cow_id] = cv2.VideoWriter(cow_out_path, fourcc, fps, target_size)

                cow_writers[cow_id].write(padded_crop)

    cap.release()
    for writer in cow_writers.values():
        writer.release()

    print("Padded cow videos saved.")


In [6]:
extract_cow_crops_padded(
    video_path="clips/001_250330120000_6001_FF_00_59_row1_clip1_Feeding.mp4",
    label_dir="track_results/001_250330120000_6001_FF_00_59_row1_clip1_Feeding/labels",  # From ByteTrack
    output_dir="cow_crops_w_padding/001_250330120000_6001_FF_00_59_row1_clip1_Feeding",
     target_size=(640, 640)
)

Video: clips/001_250330120000_6001_FF_00_59_row1_clip1_Feeding.mp4 | FPS: 30, Width: 1920, Height: 1080
Padded cow videos saved.


In [3]:
# === Batch Run for All Videos in 'clips/' ===

clips_dir = "clips4"                        # Folder with input .mp4 files
track_root = "track_results4"               # Folder with tracking results
output_root = "PaddedCowVideos4"            # Folder to save cow-wise videos

os.makedirs(output_root, exist_ok=True)

for filename in os.listdir(clips_dir):
    if filename.endswith(".mp4"):
        video_name = os.path.splitext(filename)[0]
        video_path = os.path.join(clips_dir, filename)
        label_dir = os.path.join(track_root, video_name, "labels")
        output_dir = os.path.join(output_root, video_name)

        if not os.path.exists(label_dir):
            print(f"[WARNING] Skipping {video_name}: no label folder found.")
            continue

        print(f"\n>>> Processing: {video_name}")
        try:
            extract_cow_crops_padded(video_path, label_dir, output_dir, target_size=(640, 640))
        except Exception as e:
            print(f"[ERROR] Failed on {video_name}: {e}")



>>> Processing: 001_250323190000_6001_FF_00_59_row2_00-00-20_10s
Video: clips4/001_250323190000_6001_FF_00_59_row2_00-00-20_10s.mp4 | FPS: 30, Width: 1920, Height: 1080
Padded cow videos saved.

>>> Processing: 001_250323190000_6001_FF_00_59_row31_00-25-20_10s
Video: clips4/001_250323190000_6001_FF_00_59_row31_00-25-20_10s.mp4 | FPS: 30, Width: 1920, Height: 1080
Padded cow videos saved.

>>> Processing: 001_250323190000_6001_FF_00_59_row25_00-24-10_10s
Video: clips4/001_250323190000_6001_FF_00_59_row25_00-24-10_10s.mp4 | FPS: 30, Width: 1920, Height: 1080
Padded cow videos saved.

>>> Processing: 001_250323190000_6001_FF_00_59_row13_00-04-08_10s
Video: clips4/001_250323190000_6001_FF_00_59_row13_00-04-08_10s.mp4 | FPS: 30, Width: 1920, Height: 1080
Padded cow videos saved.

>>> Processing: 001_250323190000_6001_FF_00_59_row6_00-01-10_10s
Video: clips4/001_250323190000_6001_FF_00_59_row6_00-01-10_10s.mp4 | FPS: 30, Width: 1920, Height: 1080
Padded cow videos saved.

>>> Processing: 00

In [1]:
import os

folder_path = "Annotated"  # Or any top-level folder
total_files = 0

for root, dirs, files in os.walk(folder_path):
    total_files += len([f for f in files if not f.startswith('.')])  # skip hidden files

print(f"Total files in '{folder_path}' and subfolders: {total_files}")


Total files in 'Annotated' and subfolders: 1026
